In [0]:
import numpy as np
import pandas as pd
!pip install xlrd

In [0]:
from google.colab import files

uploaded = files.upload()

Saving Data Thailand HR_analysis.xlsx to Data Thailand HR_analysis.xlsx


In [0]:
data_file = pd.ExcelFile('Data Thailand HR_analysis.xlsx')

mainDF = pd.read_excel(data_file, 'For Model_to export_v3')
edu_Allowance = pd.read_excel(data_file, 'Education allowance')
education = pd.read_excel(data_file, 'Education')

In [0]:
mainDF.drop([ 'Gender', 'Supervisor Serial', 'SUP-Gender (before)',
       'SUP-Gender', 'Re-Entry?', 'Active?', 'MPT?', 'Ed Allowance?',
       'House Loan?', 'Resign?', 'Birthday', 'Entry Date', 'Action Date',
       'SUP-Birthday', 'SUP-Entry Date', 'SUP-Action Date', 'SUP-Active?',
       'Earliest Action Date', 'Age @Action','SUP-Age @Action',
       'SUP-Exp @Action'], axis=1, inplace=True)

In [0]:
rename_col = {
    'Exp @Action':'EmpExp'
}
mainDF.rename(rename_col, axis=1, inplace=True)

In [150]:
mainDF.head()

,Serial Number,EmpExp
0,1,43.778082
1,2,38.356164
2,3,39.654795
3,4,38.273973
4,5,39.104110


In [0]:
listOfEduUsed = edu_Allowance['Serial Number'].unique()

In [0]:
mainDF['isEduAllowanceUsed'] = np.where(mainDF['Serial Number'].isin(listOfEduUsed),1,0)

In [0]:
mainDF['NewAndNotTakeMaster'] = np.where((mainDF['EmpExp'] <5) & (mainDF['isEduAllowanceUsed'] ==0),1,0)

In [154]:
mainDF.head()

,Serial Number,EmpExp,isEduAllowanceUsed,NewAndNotTakeMaster
0,1,43.778082,0,0
1,2,38.356164,0,0
2,3,39.654795,0,0
3,4,38.273973,0,0
4,5,39.104110,0,0


In [0]:
import re

In [0]:
rename_col = {
    'Institute/location':'Institute'
}
education.rename(rename_col, axis=1, inplace=True)


In [157]:
education.head()

,Serial Number,Start Date (0022),End Date (0022),Institute,Certificate Text,Branch of study 1 Text
0,1,1950-01-01,1953-01-01 00:00:00,SUANKWONG SCH.,Attended,General Studies
1,2,1953-01-01,1957-01-01 00:00:00,VIENGPROU VITHAYA SCH.,Attended,No Specific Branch
2,3,1949-01-01,1952-01-01 00:00:00,WADTEPSIRIN SCH.,Attended,No Specific Branch
3,4,1956-01-01,1959-01-01 00:00:00,WATTANA VITHAYA ACADEMY.,Attended,Education
4,5,1956-06-01,1960-03-31 00:00:00,THAMMASAT UNIVERSITY,Bachelor,Business-Economics


In [0]:
education.drop(['Start Date (0022)', 'End Date (0022)'], axis=1,inplace=True)

In [0]:
edu_patterns = {
    'UNIV' : r'UNIV\.|\WU\.|^U\.|UNIV|ERSITY|UNI$',
    'SCHL' : r'SCH\S|\WSCH$|SUKSA',
    'INST' : r'\WINST|^INST',
    'COLL' : r'\WCOLL|^COLL|COL$|COLLEGE',
    'CAMP' : r'C..PUS',
    'ACAD' : r'ACAD',
    'NIDA' : r'NIDA',
    'SASI' : r'SASIN',
}

for name, patt in edu_patterns.items():
    education.loc[education['Institute'].str.contains(patt), 'Max_EduInstituteGroup'] = name
    
    education['Max_EduInstituteGroup'].fillna('OTHR', inplace=True)

In [0]:
def remove_Suffix(institute):
        edu_patterns = {
            'UNIV' : r'(\_a)?UNIV\.|\WU\.|^U\.|(\_a)?UNIV|(\_a)?ERSITY|(\_a)?UNI$',
            'SCHL' : r'SCH\S|\WSCH$|SUKSA',
            'INST' : r'\WINST|^INST',
            'COLL' : r'\WCOLL|^COLL|COL$|COLLEGE',
            'CAMP' : r'C..PUS',
            'ACAD' : r'ACAD',
            'UNIV' : r'NIDA',
            'UNIV' : r'SASIN',
        }
        for name, patt in edu_patterns.items():
              #temp = institute.replace(patt, '')
          regexp = re.compile(patt)
          if(regexp.search(institute)):
              temp = re.sub(patt,'',institute)
              return temp;

In [0]:
education['Institute'] = education['Institute'].apply(remove_Suffix)

In [169]:
education.head()

,Serial Number,Institute,Certificate Text,Branch of study 1 Text,Max_EduInstituteGroup
0,1,SUANKWONG,Attended,General Studies,SCHL
1,2,VIENGPROU VITHAYA,Attended,No Specific Branch,SCHL
2,3,WADTEPSIRIN,Attended,No Specific Branch,SCHL
3,4,WATTANA VITHAYA EMY.,Attended,Education,ACAD
4,5,None,Bachelor,Business-Economics,UNIV


In [0]:
import string

In [0]:
def removeDot(institute):
  return institute.replace('.','')

In [164]:
education['Institute'] = education['Institute'].replace({'\.':''})

education.head(15)

,Serial Number,Start Date (0022),End Date (0022),Institute,Certificate Text,Branch of study 1 Text,Max_EduInstituteGroup
0,1,1950-01-01,1953-01-01 00:00:00,SUANKWONG,Attended,General Studies,SCHL
1,2,1953-01-01,1957-01-01 00:00:00,VIENGPROU VITHAYA,Attended,No Specific Branch,SCHL
2,3,1949-01-01,1952-01-01 00:00:00,WADTEPSIRIN,Attended,No Specific Branch,SCHL
3,4,1956-01-01,1959-01-01 00:00:00,WATTANA VITHAYA EMY.,Attended,Education,ACAD
4,5,1956-06-01,1960-03-31 00:00:00,None,Bachelor,Business-Economics,UNIV
5,6,1949-01-01,1953-01-01 00:00:00,WADCHAENG,Attended,No Specific Branch,SCHL
6,7,1959-06-01,1961-08-30 00:00:00,TECHNICAL TRAINING OL,Diploma,Other Degree,SCHL
7,8,1956-01-01,1959-01-01 00:00:00,BANGKOK COMMERCIALEGE,Attended,Finance/Accounting,COLL
8,8,1958-01-01,1961-01-01 00:00:00,BANGKOK TECHNICALITUTE,Attended,Finance/Accounting,INST
9,9,1944-01-01,1948-01-01 00:00:00,RONGKAE,Attended,No Specific Branch,SCHL


In [165]:
education[(education['Max_EduInstituteGroup'] == 'UNIV')].Institute

4       None
12      None
19      None
21      None
25      None
26      None
27      None
28      None
33      None
36      None
37      None
38      None
39      None
49      None
100     None
101     None
108     None
112     None
131     None
139     None
140     None
144     None
147     None
151     None
153     None
162     None
168     None
177     None
178     None
180     None
        ... 
7297    None
7298    None
7299    None
7300    None
7301    None
7302    None
7303    None
7304    None
7305    None
7307    None
7308    None
7309    None
7310    None
7311    None
7312    None
7313    None
7314    None
7316    None
7317    None
7318    None
7319    None
7320    None
7321    None
7322    None
7323    None
7324    None
7325    None
7326    None
7328    None
7329    None
Name: Institute, Length: 5191, dtype: object

In [0]:
newDF = pd.merge(mainDF, education, on='Serial Number')

In [172]:
newDF.head()

,Serial Number,EmpExp,isEduAllowanceUsed,NewAndNotTakeMaster,Institute,Certificate Text,Branch of study 1 Text,Max_EduInstituteGroup
0,1,43.778082,0,0,SUANKWONG,Attended,General Studies,SCHL
1,2,38.356164,0,0,VIENGPROU VITHAYA,Attended,No Specific Branch,SCHL
2,3,39.654795,0,0,WADTEPSIRIN,Attended,No Specific Branch,SCHL
3,4,38.273973,0,0,WATTANA VITHAYA EMY.,Attended,Education,ACAD
4,5,39.104110,0,0,None,Bachelor,Business-Economics,UNIV


In [0]:
newDF.to_csv('Additional_Education.csv')

In [0]:
from google.colab import files
files.download('Additional_Education.csv')